In [4]:
import pandas as pd
import spacy
import glob
import os
from collections import Counter 
import numpy as np
from spacy.lang.en import English
from spacy.lookups import Lookups
from spacy.pipeline import EntityRuler
import json

In [50]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


all_files = glob.glob("../data/*.json")
print(all_files)

li = []
for file in all_files:
    data = pd.read_json(file)
    li.append(data)

temp = pd.concat(li, ignore_index=True)
df = temp[["tweet_id", "tweet_text"]]
print(df)

d_types = load_data("../data/disasters/disaster_types.json")

['../data/california_wildfires_final_data.json', '../data/hurricane_harvey_final_data.json', '../data/hurricane_irma_final_data.json', '../data/hurricane_maria_final_data.json', '../data/iraq_iran_earthquake_final_data.json', '../data/mexico_earthquake_final_data.json', '../data/srilanka_floods_final_data.json']
                 tweet_id                                         tweet_text
0      917791044158185472  RT @Gizmodo: Wildfires raging through Northern...
1      917791130590183424      PHOTOS: Deadly wildfires rage in California  
2      917791291823591424  RT @Cal_OES: PLS SHARE: Were capturing wildfir...
3      917791291823591424  RT @Cal_OES: PLS SHARE: Were capturing wildfir...
4      917792092100988928  RT @TIME: California's raging wildfires as you...
...                   ...                                                ...
18077  881714673157980160                Get to mora and try the whole menu 
18078  881749861871362048  Mirka Mora (b.1928), noted French-born Aust

In [46]:
nlp = spacy.load("en_core_web_sm", disable=["parser"])

def create_training_data(file, type):
    data = load_data(file)
    disasters = data["disasters"]
    patterns = []
    for item in disasters:
        doc = nlp(item)
        #print(item)
        pattern_tokens = [{"LEMMA": token.lemma_.lower()} for token in doc]
        #print(pattern_tokens)
        
        patterns.append({"label": type, "pattern": pattern_tokens})
    return patterns

# {"label": "DISASTER", "pattern": [{"lemma": "wildfire"}]}

def generate_rules_n_lemma(patterns):
    ruler = nlp.add_pipe("entity_ruler", before="ner")
    ruler.add_patterns(patterns)
    
    print(f"Total patterns added: {len(ruler.patterns)}")
    print(f"Sample patterns: {ruler.patterns[:5]}")  # Print first 5 to verify
    
    nlp.to_disk("disaster_ner")

#patterns = create_training_data("../data/disasters/disaster_types.json", "DISASTER")
#generate_rules_n_lemma(patterns)
        #Rules already generated (check app folder)

FileNotFoundError: [Errno 2] No such file or directory: '../data/disaster_types.json'

In [68]:
nlp = spacy.load("disaster_ner")
def test_model(text):
    doc = nlp(text.lower())
    for ent in doc.ents:
        print(f"Entity: {ent.text} | Label: {ent.label_}")
print(df1.loc[0, "tweet_text"])

test_model("RT @Cal_OES: PLS SHARE: Were capturing Wildfire response, recovery info here:")
test_model("PHOTOS: Deadly wildfires rage in California and Mexico")
test_model(df1.loc[0, "tweet_text"])
test_model("Huge tidal waves hit Tokyo")


      

RT @Gizmodo: Wildfires raging through Northern California are terrifying  
Entity: wildfire | Label: DISASTER
Entity: wildfires | Label: DISASTER
Entity: california | Label: GPE
Entity: mexico | Label: GPE
Entity: wildfires | Label: DISASTER
Entity: northern california | Label: LOC
Entity: tidal waves | Label: DISASTER
Entity: tokyo | Label: GPE


In [62]:
def extract_entities(text):
    print(text)
    doc = nlp(text.lower())
    disasters = []
    locations = []

    for ent in doc.ents:
        if ent.label_ == "DISASTER":
            disasters.append(ent.text)
        elif ent.label_ in ["GPE", "LOC", "FAC"]:
            locations.append(ent.text)

    return {"disasters": disasters, "locations": locations}

df.loc[:, ["disasters", "locations"]] = df["tweet_text"].apply(
    lambda x: pd.Series(extract_entities(x))
)

RT @Gizmodo: Wildfires raging through Northern California are terrifying  
PHOTOS: Deadly wildfires rage in California  
RT @Cal_OES: PLS SHARE: Were capturing wildfire response, recovery info here:  
RT @Cal_OES: PLS SHARE: Were capturing wildfire response, recovery info here:  
RT @TIME: California's raging wildfires as you've never seen them before  
Wildfires Threaten Californias First Legal Cannabis Harvest  
Mass Evacuations in California as Wildfires Kill at Least 10  #CaliforniaWildfires 
RT @KAKEnews: California wildfires destroy more than 50 structures:  #KAKEnews 
RT @KAKEnews: California wildfires destroy more than 50 structures:  #KAKEnews 
RT @KAKEnews: California wildfires destroy more than 50 structures:  #KAKEnews 
RT @FoxNews: Southern California fire shrouds Disneyland Anaheim in dramatic, smoky skies  
California wildfire. 4 
Wildfires Still Burn in Northern California; 11 Are Dead  #nyt 
Henry Cejudo suffers burns, loses gold medal in California wildfire   
At Leas

In [67]:
exploded = df.explode("disasters").explode('locations').dropna()
crisis_counts = (
    exploded.groupby(["disasters", "locations"])
    .size()
    .reset_index(name="count")
    .sort_values("count", ascending=False)
)

print(crisis_counts)


               disasters       locations  count
101           earthquake          mexico    656
652            wildfires      california    580
72            earthquake            iran    381
73            earthquake            iraq    263
306            hurricane         florida    206
..                   ...             ...    ...
6             brush fire      zuma beach      1
10    buildings collapse     mexico city      1
11   buildings collapsed          mexico      1
12       chemical spills           texas      1
13               cyclone  @nwslittlerock      1

[696 rows x 3 columns]


In [144]:
#checking all entities that were found
def extract_all_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        print(f"{i} -> {ent.text}, {ent.label_} \n")

for i in range(len(df1)):
    extract_all_entities(df1.loc[i, "tweet_text"])

0 -> RT @Gizmodo, ORG 

0 -> Northern California, LOC 

1 -> wildfires, DISASTER 

1 -> California, GPE 

2 -> PLS SHARE, ORG 

2 -> wildfire, DISASTER 

3 -> PLS SHARE, ORG 

3 -> wildfire, DISASTER 

4 -> RT @TIME, PERSON 

4 -> California, GPE 

4 -> wildfires, DISASTER 

